##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrar el almacenamiento del punto de verificación

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/checkpoint_saver">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/checkpoint_saver.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/checkpoint_saver.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a></td>
</table>


Guardar continuamente el "mejor" modelo o los pesos/parámetros del modelo tiene muchas ventajas. Por ejemplo, permite realizar un seguimiento del progreso del entrenamiento y cargar modelos guardados a partir de diferentes estados guardados.

En TensorFlow 1, para configurar guardar puntos de verificación durante el entrenamiento/validación con las APIs `tf.estimator.Estimator` se especifica una programación en `tf.estimator.RunConfig` o se utiliza `tf.estimator.CheckpointSaverHook`. Esta guía muestra cómo migrar de este flujo de trabajo a las APIs de Keras de TensorFlow 2.

En TensorFlow 2, puede configurar `tf.keras.callbacks.ModelCheckpoint` de varias maneras:

- Guarda la "mejor" versión según una métrica monitorizada mediante el parámetro `save_best_only=True`, donde `monitor` puede ser, por ejemplo, `'loss'`, `'val_loss'`, `'accuracy', or `'val_accuracy'`.
- Guardar continuamente a una frecuencia determinada (utilizando el argumento `save_freq`).
- Guarde solo los pesos/parámetros en vez de todo el modelo definiendo `save_weights_only` como `True`.

Para obtener más información, consulte los documentos de la API `tf.keras.callbacks.ModelCheckpoint` y la sección *Guardar puntos de verificación durante el entrenamiento* del tutorial [Guardar y cargar modelos](../../tutorials/keras/save_and_load.ipynb). Aprenda más sobre el formato de puntos de verificación en la sección *Formato de puntos de verificación* de la guía [Guardar y cargar modelos de Keras](https://www.tensorflow.org/guide/keras/save_and_serialize). Además, para agregar tolerancia ante errores, puede utilizar `tf.keras.callbacks.BackupAndRestore` o `tf.train.Checkpoint` para el punto de verificación manual. Obtenga más información en la [Guía de migración de tolerancia ante errores](fault_tolerance.ipynb).

Las [retrollamadas](https://www.tensorflow.org/guide/keras/custom_callback) de Keras son objetos que se llaman en diferentes puntos durante el entrenamiento/evaluación/predicción en las APIs incorporadas de Keras `Model.fit`/`Model.evaluate`/`Model.predict` de las API. Obtenga más información en la sección *Siguientes pasos* al final de la guía.

## Preparación

Empiece con imports y un conjunto de datos sencillo a modo de demostración:

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import numpy as np
import tempfile

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## TensorFlow 1: Guardar puntos de verificación con el tf.estimator de las API

Este ejemplo de TensorFlow 1 muestra cómo configurar `tf.estimator.RunConfig` para guardar puntos de verificación en cada paso durante el entrenamiento/evaluación con las `tf.estimator.Estimator` de las API:

In [ ]:
feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

config = tf1.estimator.RunConfig(save_summary_steps=1,
                                 save_checkpoints_steps=1)

path = tempfile.mkdtemp()

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.2,
    model_dir=path,
    config = config
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

In [ ]:
%ls {classifier.model_dir}

## TensorFlow 2: Guardar puntos de verificación con una retrollamada de Keras para Model.fit

En TensorFlow 2, cuando utilice el método incorporado de Keras `Model.fit` (o `Model.evaluate`) para entrenamiento/evaluación, puede configurar `tf.keras.callbacks.ModelCheckpoint` y luego pasarlo al parámetro `callbacks` de `Model.fit` (o `Model.evaluate`). (Obtenga más información en los documentos de la API y en la sección *Uso de retrollamadas* de la guía [Entrenamiento y evaluación con los métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate)).

En el siguiente ejemplo, se utilizará una retrollamada `tf.keras.callbacks.ModelCheckpoint` para almacenar los puntos de verificación en un directorio temporal:

In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              steps_per_execution=10)

log_dir = tempfile.mkdtemp()

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=log_dir)

model.fit(x=x_train,
          y=y_train,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[model_checkpoint_callback])

In [ ]:
%ls {model_checkpoint_callback.filepath}

## Siguientes pasos

Obtenga más información sobre los puntos de verificación en:

- Documentos de la API: `tf.keras.callbacks.ModelCheckpoint`
- Tutorial: [Guardar y cargar modelos](../../tutorials/keras/save_and_load.ipynb) (la sección *Guardar puntos de verificación durante el entrenamiento*)
- Guía: [Guardar y cargar modelos de Keras](https://www.tensorflow.org/guide/keras/save_and_serialize) (la sección *Formato de punto de verificación TF*)

Obtenga más información sobre las retrollamadas en:

- Documentos de la API: `tf.keras.callbacks.Callback`
- Guía: [Escribiendo sus propias retrollamadas](https://www.tensorflow.org/guide/keras/guide/keras/custom_callback)
- Guía: [Entrenamiento y evaluación con los métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (la sección *Usar retrollamadas*)

También le pueden resultar útiles los siguientes recursos relacionados con la migración:

- La [Guía de migración de tolerancia ante errores](fault_tolerance.ipynb): `tf.keras.callbacks.BackupAndRestore` para `Model.fit`, o `tf.train.Checkpoint` y `tf.train.CheckpointManager` de las API para un bucle de entrenamiento personalizado
- La [Guía de migración de parada anticipada](early_stopping.ipynb): `tf.keras.callbacks.EarlyStopping` es una retrollamada de parada anticipada incorporada.
- La [Guía de migración de TensorBoard](tensorboard.ipynb): TensorBoard permite el seguimiento y la visualización de métricas
- La [Guía de migración de retrollamadas LoggingTensorHook y StopAtStepHook a Keras](logging_stop_hook.ipynb)
- La [Guía de retrollamadas de SessionRunHook a Keras](sessionrunhook_callback.ipynb)